# Using Dynamic IP on the Composable Pipeline
----

<div class="alert alert-box alert-info">
Please use Jupyter labs http://&lt;board_ip_address&gt;/lab for this notebook.
</div>

This notebook shows your how to load dynamic IP and compose branched pipelines

## Aims
* Load dynamic IP
* Compose branched pipelines

## Table of Contents
* [Download Composable Overlay](#download)
* [Start HDMI Video](#start_hdmi)
* [Load Dynamic IP](#dynamic)
* [Let us Compose](#compose)
* [Branched Pipeline](#branched)
* [Conflicting Dynamic IP](#conflicting)
* [Stop HDMI Video](#stop_hdmi)
* [Conclusion](#conclusion)

----

## Revision History

* v1.0 | 30 March 2021 | First notebook revision.

----

## Download Composable Overlay <a class="anchor" id="download"></a>

Import the pynq video libraries as well as ComposableOverlay class and the drivers for the IP.

Download the Composable Overlay using the `ComposableOverlay` and grab a handler to the `composable` hierarchy

In [ ]:
from pynq.lib.video import *
from composable_pipeline import ComposableOverlay
from composable_pipeline.libs import *

ol = ComposableOverlay("../overlay/cv_dfx_4_pr.bit")

cpipe = ol.composable

## Start HDMI Video <a class="anchor" id="start_hdmi"></a>

Get `HDMIVideo` object and start video

<div class="alert alert-heading alert-danger">
    <h4 class="alert-heading">Warning:</h4>

Failure to connect HDMI cables to a valid video source and screen may cause the notebook to hang
</div>

In [ ]:
video = HDMIVideo(ol)
video.start()

## Load Dynamic IP <a class="anchor" id="dynamic"></a>

The Composable Overlay provides DFX regions where IP can be loaded dynamically to bring new functionality. If we want to load an IP within a DFX region, the `.loadIP` method is used. 

Let us start by looking at the `.c_dict` to see what IP cores are loaded

In [ ]:
cpipe.c_dict.loaded

The documentation of `.loadIP` specify that IP can be loaded using the full name or the IP object

In [ ]:
cpipe.loadIP?

In [ ]:
cpipe.loadIP([cpipe.pr_1.dilate_accel])

Examine the `.c_dict` again and verify that `dilate_accel` and `erode_accel` are indeed loaded, both are in the same DFX region

In [ ]:
cpipe.c_dict.loaded

## Let us Compose <a class="anchor" id="compose"></a>

Grab handlers to these dynamic IP objects and compose a pipeline with them

In [ ]:
video_in_in = cpipe.video.hdmi_in.color_convert
video_in_out = cpipe.video.hdmi_in.pixel_pack

dilate = cpipe.pr_1.dilate_accel

In [ ]:
cpipe.compose([video_in_in, dilate, video_in_out])

cpipe.graph

## Branched Pipeline <a class="anchor" id="branched"></a>

In this part of the notebook, we will bring new functionality into the four DFX regions to compose the [Difference of Gaussians](https://en.wikipedia.org/wiki/Difference_of_Gaussians) application that was also introduced in the previous session. 

Load dynamic IP, grab handlers and set up default values

In [ ]:
cpipe.loadIP(['pr_fork/duplicate_accel', 'pr_join/subtract_accel', 'pr_0/filter2d_accel'])

In [ ]:
filter2d = cpipe.video.composable.filter2d_accel
duplicate = cpipe.pr_fork.duplicate_accel
subtract = cpipe.pr_join.subtract_accel
fifo = cpipe.pr_0.axis_data_fifo_1
filter2d_d = cpipe.pr_0.filter2d_accel

filter2d.sigma = 0.3
filter2d.kernel_type = 'gaussian_blur'

filter2d_d.sigma = 12
filter2d_d.kernel_type = 'gaussian_blur'

The Difference of Gaussians is realized by subtracting one Gaussian blurred version of an original image from another less blurred version of the original. In the Composable Overlay this is achieved by branching the pipeline, which is expressed as a list of a list.

In [ ]:
video_pipeline = [video_in_in, filter2d, duplicate, [[filter2d_d], [1]], subtract, video_in_out]

cpipe.compose(video_pipeline)

cpipe.graph

## Conflicting Dynamic IP <a class="anchor" id="conflicting"></a>

Note that IP within the DFX regions are often mutually exclusive (some DFX regions support multiple IP at the same time), this means that they cannot be loaded at the same time. The `.loadIP` will raise an exception in these cases, try it by yourself running the following cell

In [ ]:
cpipe.loadIP(['pr_fork/duplicate_accel', 'pr_fork/colorthresholding_accel'])

<div class="alert alert-info">
  <strong>Info!</strong> Use the <strong>dfx_dict</strong> attribute to identify which IP are mutually exclusive
</div>

## Stop HDMI Video <a class="anchor" id="stop_hdmi"></a>

Finally stop the HDMI video pipeline

<div class="alert alert-heading alert-danger">
    <h4 class="alert-heading">Warning:</h4>

Failure to stop the HDMI Video may hang the board 
when trying to download another bitstream onto the FPGA
</div>

In [ ]:
video.stop()

----

## Conclusion <a class="anchor" id="conclusion"></a>

This notebook has shown how to bring new functionality to the composable overlay by loading dynamic IP. Moreover, the notebook shows how to implement a branched pipeline.

[⬅️ Modify Composable Pipeline](04_modify_pipeline.ipynb) | | [Build Custom Application ➡️](06_build_application.ipynb)

Copyright &copy; 2021 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

----